## How to break an Enigma machine like Rejewski

This notebook will walk you through the process of cracking an Engima machine. 

Before you get started, here's what you need to know: 
  - You will be working with the 3 rotor Wehrmacht Enigma machine. Read more about it [here](http://users.telenet.be/d.rijmenants/en/enigmatech.htm). You will only consider rotors I, II, and III for the purposes of this exercise. 
  - The dictionary of cycle lengths has already been generated for you. You will access it later. 
  - There's a little bit of historical anachronism in the scenario described, since Rejewski's method of cracking Enigma was developed long before Bletchley Park existed, and codebreakers at Bletchley Park used more advanced methods than did Rejewski. Just roll with it. 

### The Scenario. 

It's midnight at Bletchley, the middle of your nighttime shift. The past days of German army traffic you decrypted have hinted at some major events to come today. Since the new day just started, you know that the German army Enigma operators have just switched over to a new day key, rendering the previous day's work decrypting their traffic effectively useless. Given the rumors swirling about a potential German attack today, you know you need to crack this new day key as soon as possible. 

Shortly after midnight, freshly encrypted German messages begin piling up on your desk, demanding your attention. You know what to do. Let's get cracking. 

### Materials

You have at your hands the following materials: 
- A huge set of German messages encrypted with the unknown day key (which includes the rotor key, plugboard settings, and rotor order). They are stored in the file 'message_key_encrypts.pickle'. For your conveniences, only the first 6 letters from each of these encrypts (the twice-enciphered message key) have been kept. 
- A dictionary of all the different chain lengths generated by every possible rotor key and order combination. 
- A small test set of messages encrypted with the day key. When you decrypt these properly, you will see English text and will have confirmation that you have correctly guessed the day key. 
- Also you have a computer. You can certainly do parts of this exercise manually, but please please feel free to write some code to supplement your work. After all, breaking Enigma took Rejewski over a year. You have an hour and a half. 

### A Brief Enigma Overview
The below code will walk you through how to use the Enigma simulator written for this exercise. This knowledge should help you as you figure out how to crack Enigma.

In [1]:
# Import the Enigma module. 
# REMEMBER: to run a particular cell in a Jupyter notebook, press Ctrl + Enter.
import machine as m

In [2]:
# Create an Enigma machine. 
# Since you don't specify any settings, this Enigma is created with the initial rotor position setting 'AAA'
# and rotor order I, II, III.
e = m.Enigma()

In [3]:
# Set the Enigma machine rotor position so that you are sure it is initialized for encryption. 
e.set_rotor_position('AAA')
# Now encrypt.
e.encipher('this is a test')

'ZPJJSVSPGBW'

In [4]:
# Now reset the rotor position so that it is initialized for decryption. 
e.set_rotor_position('AAA')
# Now decrypt. 
e.decipher('zpjjsvspgbw')

'THISISATEST'

Note that you can use the e.enicpher and e.decipher interchangably, since encryption and decryption are the same. However, these two separate functions are provided for your convenience. 

### Now, let's break Enigma

In [12]:
# First, import all the modules you will need.
import machine as m
import pickle

In [13]:
# Next, load in the message key encrypts. 
message_key_encrypts = pickle.load(open('message_key_encrypts.pickle', 'rb'))
# Take a look at the first 30 message key encrypts. 
message_key_encrypts[0:50]

['ZEUATW',
 'MVBXPQ',
 'LSCEJY',
 'GYICYG',
 'ORZNBX',
 'UZBBXQ',
 'LCHEKP',
 'DMGOOO',
 'UQVBDR',
 'BPZSIX',
 'RJOFRE',
 'YCFJKC',
 'SCTTKZ',
 'HCGKKO',
 'NSVWJR',
 'ZWGAEO',
 'EBTLAZ',
 'TWRIET',
 'AIVHSR',
 'PTLUUB',
 'ZVKAPS',
 'XZNPXL',
 'SWFTEC',
 'ZSOAJE',
 'WJKZRS',
 'SSLTJB',
 'PDNUVL',
 'PNDUNA',
 'QWMMEV',
 'XYIPYG',
 'HRQKBK',
 'PGNUFL',
 'QPQMIK',
 'UTABUJ',
 'MQAXDJ',
 'EWXLEM',
 'MNFXNC',
 'MDCXVY',
 'DISOSN',
 'UWXBEM',
 'AXBHMQ',
 'EQDLDA',
 'EZPLXU',
 'BSNSJL',
 'NQLWDB',
 'GQXCDM',
 'OJFNRC',
 'JSJYJI',
 'SQBTDQ',
 'LOGEHO']

Following Rejewski's original attack, you will need to generate the AD, BE, and CF permutations. Recall that, if we think of the twice-encrypted message key as ABCDEF, we know that the first and third (A and D), second and fourth (B and E), and third and fifth (C and F) letters in the encryption are generated by the same plaintext letter. Because we know this intimate relationship between AD, BE, and CF, we can generate permutations for these three pairs. 

Take the following 5 message encrypts for example: 'LCAUAD', 'PQISES', 'HEKYGI', 'QUMORJ', and 'BCGMAG'. Because of the known relationship between the first and fourth letters (A and D) in these encrypts, we can generate the following mappings. 

- AD: L -> U, P -> S, H -> Y, Q -> O, B -> M. 

Similarly, for BE and CF, we find:
 
- BE: C -> A, Q -> E, E -> G, U -> R (we ignore the fifth because it gives us the same C -> A mapping). 
- CF: A -> D, I -> S, K -> I, G -> G (a letter mapping to itself is ok in this scenario because this is a permutation map, not a direct map). 

If we were to repeat this process until we found a mapping for every letter in the alphabet for each of the AD, BE, and CF permuations, we could figure out the "chain lengths" generated by these permutation maps. If you're fuzzy on the cycle length description, revisit slides 150-180. 

In [14]:
# So let's do it. First, generate the AD, BE, and CF permutation mappings. 
# You can do this by hand if you wish or you can write code to do so. 

# Here's code if you want it. It will return dictionaries containing the alphabet mappings for AD, BE, and CF. 
def generate_permutation_dicts(message_encrypts):
    '''
    Outputs AD, BE, and CF dictionaries generated from a series of double message key encryptions. 
    
    Params: message_encrypts = an iterator/list containing a double message key encryptions. 
    '''
    alphabet = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    AD = {letter:None for letter in alphabet}
    BE = {letter:None for letter in alphabet}
    CF = {letter:None for letter in alphabet}
    for m in message_encrypts:
         if any(AD[i] is None for i in AD) or any(BE[i] is None for i in BE) or any(CF[i] is None for i in CF):
                AD[m[0]] = m[3] if AD[m[0]] is None else AD[m[0]]
                BE[m[1]] = m[4] if BE[m[1]] is None else BE[m[1]]
                CF[m[2]] = m[5] if CF[m[2]] is None else CF[m[2]]
    return AD, BE, CF

In [15]:
AD, BE, CF = generate_permutation_dicts(message_key_encrypts)

In [16]:
BE

{'A': 'Q',
 'B': 'A',
 'C': 'K',
 'D': 'V',
 'E': 'T',
 'F': 'G',
 'G': 'F',
 'H': 'W',
 'I': 'S',
 'J': 'R',
 'K': 'C',
 'L': 'Z',
 'M': 'O',
 'N': 'N',
 'O': 'H',
 'P': 'I',
 'Q': 'D',
 'R': 'B',
 'S': 'J',
 'T': 'U',
 'U': 'L',
 'V': 'P',
 'W': 'E',
 'X': 'M',
 'Y': 'Y',
 'Z': 'X'}

In [17]:
# Now, find the cycles from the alphabet mappings. 
# Recall that these are of the form AD = (bfhjqkld)(capmvixuzr)(egnopstw)(y).

# Again, here's code if you want it. 
def make_chains_from_permutation_dict(permutation_dict):
    '''
    Given a dictionary linking the first and fourth (or second and fifth or third and sixth) 
    letter combinations from the message keys, generate the length of chains (i.e. cycles of letters) 
    found in the dictionary.
    '''
    alphabet = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    chains = []
    while len(alphabet) > 0:
        next_chain = permutation_dict[alphabet[0]]
        next_letter = permutation_dict[next_chain[0]]
        # must remove the first two letters in the chain.
        alphabet = alphabet.replace(alphabet[0], '')
        alphabet = alphabet.replace(next_chain, '')
        while next_letter != next_chain[0]:
            next_chain += next_letter
            alphabet = alphabet.replace(next_letter, '')
            next_letter = permutation_dict[next_letter]
        chains.append(next_chain)
        alphabet.replace(next_letter, '')
    return chains

In [18]:
AD_chains = make_chains_from_permutation_dict(AD)
BE_chains = make_chains_from_permutation_dict(BE)
CF_chains = make_chains_from_permutation_dict(CF)

In [19]:
AD_chains, BE_chains, CF_chains

(['HKVDONWZA', 'STIQMXPUB', 'GC', 'LE', 'RF', 'YJ'],
 ['QDVPISJRBA', 'KC', 'TULZXMOHWE', 'GF', 'N', 'Y'],
 ['JIGOEDA', 'QKSNLB', 'YHPUWFC', 'VRTZXM'])

Whew -- you've made it through a lot of hard work, and you're nearly there. With the cycles and their lengths in hand, you approach the trust catalog you spent years compiling. This catalog is indexed by cycle length and maps cycle length to putative rotor keys and order. 

The index is a little unique. It is a string of the chain lengths in numerical order prefaced by their respective dictionary name in alphabetical order. 

For example, suppose your AD mapping had cycles of length 2, 12, and 12; your BE mapping had cycles of  6, 8, and 10; and your CF mapping had cycles of length 4,6,8, and 8. The dictionary index you would search on would be 'AD:21212 BE:6810 CF:4688'. Not the prettiest, but it gets the job done. 

In [20]:
# First, let's load the chain dictionary and take a look at it. 
chains_dict = pickle.load(open('chains.pickle', 'rb'))

In [21]:
# To reference an item in the chains_dict, use the following syntax: chains_dict[index]. Let's try it below. 
chains_dict['AD:6677 BE:11221010 CF:221111']

[(('A', 'A', 'B'), ('II', 'III', 'I')),
 (('D', 'X', 'M'), ('I', 'II', 'III')),
 (('E', 'I', 'W'), ('II', 'I', 'III')),
 (('F', 'Q', 'X'), ('I', 'III', 'II')),
 (('P', 'D', 'L'), ('II', 'I', 'III')),
 (('P', 'V', 'H'), ('II', 'III', 'I')),
 (('R', 'Y', 'E'), ('II', 'I', 'III')),
 (('T', 'X', 'C'), ('II', 'I', 'III')),
 (('U', 'M', 'W'), ('I', 'II', 'III')),
 (('Z', 'S', 'A'), ('I', 'III', 'II'))]

In [22]:
# Generate the indicies for your chains based on the permutation dictionaries. 
def generate_chain_index(chains):
    '''
    Given a list of chains for the AD, BE, and CF pairs, find their lengths and put them in a digestable form for indexing.

    MAJOR ASSUMPTION: these chains will come directly from my code, so they will be in the order AD, BE, CF.
    '''
    chain_index = 'AD:'
    for i, chain_list in enumerate(chains):
        if i is 1:
            chain_index = chain_index + ' BE:'
        elif i is 2:
            chain_index = chain_index + ' CF:'
        # Sort the chains to make sure the number is unique.
        chain_list.sort(key=lambda x: len(x))
        for c in chain_list:
            chain_index = chain_index + str(len(c))
    return chain_index

In [23]:
# Generate the dictionary index from your chains and see what you get back. 
# Note that the first item you get back represents the rotor key and the second item represents the rotor order
# from left to right.
index = generate_chain_index([AD_chains, BE_chains, CF_chains])
chains_dict[index]
# len(chains_dict[index]) = 170

[(('A', 'A', 'C'), ('II', 'III', 'I')), (('Y', 'A', 'Q'), ('II', 'I', 'III'))]

### Good work -- you're almost there!
You've now pared down your search space from 105456 entries to 2. That's a massive reduction. Well done. 
Now you just need to figure out what the correct key/rotor setting is between these two and additionally figure out the plugboard settings. 

To assist you, there exist four files: encrypt1.txt, encrypt2.txt, encrypt3.txt, and encrypt4.txt. These are messages encrypted with the day key's rotor order and plugboard settings but the first message key's (the first one in the message_key_encrypts list, 'ZEUATW') rotor position, as would have been the case for a real German Enigma encryption. 

You have all the pieces available that you need. Good luck.

In [33]:
import rejewski as r

In [37]:
r.SECRET_ROTOR_ORDER

['II', 'I', 'III']

In [38]:
r.SECRET_DAY_KEY

'YAQ'

In [39]:
r.SECRET_SWAPS

[('J', 'S'), ('H', 'Y'), ('N', 'F')]

In [51]:
e = m.Enigma()
e.set_rotor_order(['II', 'I', 'III'])
e.set_rotor_position('TDS')
e.set_plugs(r.SECRET_SWAPS)

In [52]:
e1 = e.encipher(r.msg1)
#open('./encrypt1.txt', 'w').write(e1)
e1

'ZNRJYTZHJEADPAXQZBRGEJEFQWZRNCZNXKYFHWEWWSSTXMLVQQCNJIJBXCXFKFXPZFCAKVRQTQGOIXXEUOOWWQZJRXTZPOPNTUBULJERNT'

In [44]:
e.set_rotor_position('TDS')
e2 = e.encipher(r.msg2)
open('./encrypt2.txt', 'w').write(e2)

58

In [45]:
e.set_rotor_position('TDS')
e3 = e.encipher(r.msg3)
open('./encrypt3.txt', 'w').write(e3)

73

In [53]:
# Now, you can test if you found the right message key and also recover the plugboard settings. 

# This code will take in a rotor position, rotor order, and swap setting 
# (you can specify None for the swaps if you don't know them).
# This Enigma setting will be used to decrypt the three previously mentioned encrypted files. 
# It is your job to look through these potential decryptions and try to find the real day key, 
# keeping in mind that you still will need to recover the plugboard settings. 

def test_putative_key(rotor_position, rotor_order, plugs):
    '''
    Given a list of potential day key/rotor settings from the chain dictionary, try to decrypt files. 
    
    Rotor position = some three letter string i.e 'ABC'
    Rotor order = list of rotors from left to right i.e. ['I', 'III', 'II]
    Plugs = List of letters swapped (can be None or empty) i.e [] or ['AD', 'UY', 'IO']
    '''
    with open('encrypt1.txt', 'r') as f: 
        e1 = f.read()
    with open('encrypt2.txt', 'r') as f:
        e2 = f.read()
    with open('encrypt3.txt', 'r') as f:
        e3 = f.read()
    e = m.Enigma()
    e.set_rotor_order(rotor_order)
    e.set_plugs(plugs)
    e.set_rotor_position(rotor_position)
    d1 = e.decipher(e1)
    e.set_rotor_position(rotor_position)
    d2 = e.decipher(e2)
    e.set_rotor_position(rotor_position)
    d3 = e.decipher(e3)
    print('Rotor position: ' + rotor_position + ', rotor order: ' + str(rotor_order) + ', plugboard: ' + str(plugs) + ': ')
    print(d1)
    print(d2)
    print(d3)
    print('\n')

In [57]:
# Try out your potential keys.
test_putative_key('TDS', ['II', 'I', 'III'],['HY', 'JS', 'NF'])

Rotor position: TDS, rotor order: ['II', 'I', 'III'], plugboard: ['HY', 'JS', 'NF']: 
ACALMANDMODESTLIFEBRINGSMOREHAPPINESSTHANTHEPURSUITOFSUCCESSCOMBINEDWITHCONSTANTRESTLESSNESSALBERTEINSTEIN
THEWAYTOMAKEPEOPLETRUSTWORTHYISTOTRUSTTHEMERNESTHEMMINGWAY
THEFUTUREBELONGSTOTHOSEWHOBELIEVEINTHEBEAUTYOFTHEIRDREAMSELEANORROOSEVELT




Need to figure out how to walk them through the process of guessing the swaps. I think that I have that just fine. 

QUESTION: do we need a smaller selection of possible keys?